In [278]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [288]:
from group_recommender_system import *
import networkx as nx
import random

In [289]:
import importlib
importlib.reload(rs)

<module 'group_recommender_system' from 'c:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\group_recommender_system\\__init__.py'>

In [280]:
G = nx.read_gpickle("C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\vegas_graph_small.gpickle")

In [292]:
#test_user = 'YrSb656bOiCRFlAeNYqQ-Q'
#test_user = 'hGYGNZsPhcRuLgUVWR_rjQ'
test_user = random.choice(get_nodes_of_type(G, 'user'))
print("User: ", test_user)

similar_users = most_similar_users(G, test_user)
print("Similar users: ", similar_users)

recommendations = set()
for u in similar_users:
    recommendations.update(recommend_restaurants(G, u, test_user))

nbrs = G.neighbors(test_user)

for n in nbrs:
    for r in recommendations:
        print("*", G.nodes[n]['name'], "-", G.nodes[r]['name'], " --- ", restaurant_similarity(G, n, r))


User:  abSv4EROV16qmh0W-xHZAA
Similar users:  ['SrDmD3UP8QBsJnjK4iOLZg', 'DCY_bNjVLIjVJpZjrBomgw', 'Fxzv4iufRNIyPVO0XvKmWQ', 'hMwPqwbNQV94fWSBhM4QaQ', 'C349hhMXsGVdxk6u-xf6GQ', 'aJ2Bf6IDS13QjtG_30SlRw', 'AuI6Vg60wKHTX4E5nVPbHg', 'w2Ye9CYI5DM4c_gLBJGDng', 'Yw5LynmZmKjSb4cuzgHptw', 'YVQFzWm0H72mLUh-8gzd5w', 'SqjP1cV8JwDdN0K9QULzog', '9SzSlEGovHa7py8fkOvrWQ', 'abSv4EROV16qmh0W-xHZAA', 'VzdF-Eqnp3FpKb8cYWeTSw', 'u0OJZADJMM3LwZFP-k1HXg', 'QE6-YsyTxL-iTVWUfO7TWA', 'MVRaXPXr92vF_EJbdxeZLA', 'oEnqCVEUMRjTFxVDCw2A3A', 'srXJN4ATEM1n2QLRhZM6Yg', 'NGAcu7RBt_eQFKAh5PMvFg', 'XEIuzd5ayMzNlSHww1AP8w']
* Holsteins - Custom Built Pizza  ---  0.1
* Holsteins - Texas de Brazil  ---  0.1


In [295]:
#test_user = 'YrSb656bOiCRFlAeNYqQ-Q'
#test_user = 'hGYGNZsPhcRuLgUVWR_rjQ'
test_user = '_Ojr37qJFwX0zZ2S0WtVqQ' #este tem varias categorias
#test_user = random.choice(get_nodes_of_type(G, 'user'))
print("User: ", test_user, "\n")

similar_users = most_similar_users(G, test_user)
#print("Similar users: ", similar_users)

category_ratings = predict_category_ratings(G, test_user, similar_users)
print(category_ratings, "\n")

recommendations = set()
for u in similar_users:
    recommendations.update(rs.recommend_restaurants(G, u, test_user))

for rest in recommendations:
    ratinglist = {}
    for cat in G.neighbors(rest):
        if G.nodes[cat]['type'] == 'category':
            ratinglist[cat] = category_ratings[cat]
            
    rating = predict_restaurant_ratings(G, category_ratings, rest)
    print("*", G.nodes[rest]['name'], "-", rating)
    print(" ", ratinglist, "\n")

User:  _Ojr37qJFwX0zZ2S0WtVqQ 

{'Buffets': 3, 'Breakfast & Brunch': 3, 'Restaurants': 3, 'Smokehouse': 4, 'Food': 4, 'Nightlife': 3.8333333333333335, 'American (Traditional)': 3.5, 'Southern': 4.5, 'Bars': 3.8333333333333335, 'Barbeque': 4, 'Sports Bars': 3, 'Pubs': 3.5, 'American (New)': 3.6666666666666665, 'Mexican': 2, 'Delis': 3.5, 'Vegan': 3.5, 'French': 3.5, 'Coffee & Tea': 4, 'Sandwiches': 3.5, 'Cafes': 3.5, 'Donuts': 5, 'Bakeries': 4, 'Hotels & Travel': 3, 'Resorts': 3, 'Casinos': 3, 'Japanese': 3, 'Arts & Entertainment': 3, 'Fast Food': 3, 'Vegetarian': 3, 'Salad': 3, 'Caterers': 3, 'Food Delivery Services': 3, 'Personal Chefs': 3, 'Mediterranean': 3, 'Soup': 3, 'Event Planning & Services': 3, 'Gluten-Free': 3, 'Cocktail Bars': 5, 'Gastropubs': 4, 'Irish': 4} 

* Pinkbox Doughnuts - 3.8
  {'Food': 4, 'Donuts': 5, 'Bakeries': 4, 'Restaurants': 3, 'Breakfast & Brunch': 3} 

* Fulton Street Food Hall - 3.125
  {'Hotels & Travel': 3, 'Resorts': 3, 'Casinos': 3, 'Delis': 3.5, 'Jap